In [1]:
import math
import numpy as np
import pandas as pd
import nltk

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data = pd.read_csv('../data/yelp_bubble_tea_reviews_encoded.csv')
data.head()

,Unnamed: 0,review_date,review_rating,reivew,cool,funny,useful,business_id,restaurant_name,categories,...,BikeParking,PriceRange_1,PriceRange_2,PriceRange_3,PriceRange_4,PriceRange_5,Caters,OutdoorSeating,DogsAllowed,WiFi
0,0,2015-04-15 06:25:19.000000,4,I have to say I really enjoy the Boba Tea sele...,0,0,0,SVUxmYs6_TvX5kWv0ok-MA,No. 1 Boba Tea,"""Coffee & Tea, Food, Juice Bars & Smoothies, B...",...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,1,2017-10-02 02:46:08.000000,3,I appreciate that it opens earlier than all th...,0,0,0,SVUxmYs6_TvX5kWv0ok-MA,No. 1 Boba Tea,"""Coffee & Tea, Food, Juice Bars & Smoothies, B...",...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,2,2017-07-28 19:09:36.000000,5,My favorite Boba tea place in Vegas. So many c...,0,0,0,SVUxmYs6_TvX5kWv0ok-MA,No. 1 Boba Tea,"""Coffee & Tea, Food, Juice Bars & Smoothies, B...",...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,3,2014-06-28 01:52:37.000000,5,"Love, love, love their boba and the variety of...",1,0,1,SVUxmYs6_TvX5kWv0ok-MA,No. 1 Boba Tea,"""Coffee & Tea, Food, Juice Bars & Smoothies, B...",...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,4,2015-01-04 02:12:56.000000,2,what the heck?? came here because QQ Boba was ...,0,0,1,SVUxmYs6_TvX5kWv0ok-MA,No. 1 Boba Tea,"""Coffee & Tea, Food, Juice Bars & Smoothies, B...",...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [3]:
data = data[~data['categories'].str.contains('Vietnamese')]
print(data.shape)

(37565, 43)


In [4]:
reviewsList = list(data['reivew'])
reviewsList

['I have to say I really enjoy the Boba Tea selection at this place. They have a lot of flavors and I always leave with a smile on my face. :) Yummy!',
 "I appreciate that it opens earlier than all the other places. pretty busy when they open, but you don't have to wait long ! I recommend the Hokkaido milk tea, matcha, and papaya coconut !",
 'My favorite Boba tea place in Vegas. So many choices to choose from but I always enjoy my favorite, the Lychee Black Tea!',
 "Love, love, love their boba and the variety of flavored teas and smoothies! If you're not a boba fan they have other things you can add to your drink like lychee jellies. \n\nThey are very well priced compared to other places. You can order a 32 oz. for just $5 which is pretty reasonable. What makes things even better is the fresh fruit they use for their smoothies so there is no powdery taste. Their food is also good too, but I mainly stop by for their boba. I definitely recommend this place.",
 'what the heck?? came here

In [5]:
goodReviews = data[data['review_rating'] >= 4]
badReviews = data[data['review_rating'] <= 2]

In [6]:
goodReviewsList = list(goodReviews['reivew'])
badReviewsList = list(badReviews['reivew'])

#### Limmatization

In [7]:
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatization(reviews):   # lemmatizing
    sentence =[]
    for line in reviews:
        token_words=word_tokenize(line)

        lemma=[]
        for word in token_words:
            lemma.append(lemmatizer.lemmatize(word))
            lemma.append(" ")
        sentence.append("".join(lemma))
    return sentence

In [8]:
lemma_reviewsGood = lemmatization(goodReviewsList)
lemma_reviewsBad = lemmatization(badReviewsList)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

import nltk
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')

vectorizer = CountVectorizer(token_pattern=r'\b[a-zA-Z]{3,}\b',   # three or more alpha characters
                             max_df=0.3,
                             min_df=5, 
                             stop_words=stopwords) 
vector = vectorizer.fit(lemma_reviewsGood)
vector = vectorizer.fit_transform(lemma_reviewsGood)

In [10]:
#### good reviews
corpus_df_good = pd.DataFrame(vector.toarray(), columns = vectorizer.get_feature_names())
corpus_df_good.head()

,ability,able,absence,absolute,absolutely,absurd,abundance,abundant,acai,accent,...,yummmmmm,yummmy,yummy,yup,yuzu,zen,zenq,zero,zest,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
vector = vectorizer.fit(lemma_reviewsBad)
vector = vectorizer.fit_transform(lemma_reviewsBad)

In [12]:
#### bad reviews
corpus_df_bad = pd.DataFrame(vector.toarray(), columns = vectorizer.get_feature_names())
corpus_df_bad.head()

,able,absolute,absolutely,absurd,abundance,abysmal,accent,accept,acceptable,accepted,...,york,young,younger,yuck,yum,yummy,yuzu,zen,zero,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
goodreviewsFreq = pd.DataFrame(corpus_df_good.sum(axis = 0), columns = ['frequency'])
goodreviewsFreq = goodreviewsFreq.sort_values(by='frequency',ascending=False)
# goodreviewsFreq.sort_values(by='frequency',ascending=False).to_csv('goodreviewsFreq.csv')
goodreviewsFreq.head(50)

,frequency
milk,13956
like,11044
great,10004
time,9527
get,9111
ice,8836
one,8257
love,8111
also,7999
really,7980


In [14]:
badreviewsFreq = pd.DataFrame(corpus_df_bad.sum(axis = 0), columns = ['frequency'])
badreviewsFreq = badreviewsFreq.sort_values(by='frequency',ascending=False)
# badreviewsFreq.sort_values(by='frequency',ascending=False).to_csv('badreviewsFreq.csv')
badreviewsFreq.head(50)

,frequency
order,2647
milk,2530
good,2108
ordered,2066
get,2006
would,2004
back,1993
food,1978
ice,1700
even,1686


#### Stopwords for products

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

import nltk
from nltk.corpus import stopwords
set(stopwords.words('english'))
    
stopwords = nltk.corpus.stopwords.words('english')
newStopWords = ['wa','one','get','would','place','also','good','great','love','come',\
                'definitely','nice','always','make','make','best','come','came','much','super',\
                'could','know','even','got','went','said','really','give','able','could','made','back',\
                'service','customer','great','time','get','one','love','also','really','try','flavor',\
                'go','best','delicious','go','staff','well','even',\
                'kung','fu','brew','bar',\
                ".",'.', ",",":", "''", "'s", "'", "``", "(", ")", "-"]
stopwords.extend(newStopWords)   # adding custom stopwords

#### TF-IDF

In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(3,4),
#                             token_pattern=r'\b[a-zA-Z]{3,}\b',   # three or more alpha characters
                             max_df=0.5,   # ignore appear in more than 30% of documents
                             min_df=5,     # ignore appear in less than 5 documents
                             stop_words=stopwords,
                             max_features=300)

For **good** reviews:

In [17]:
X = vectorizer.fit_transform(lemma_reviewsGood)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
tf_idf = tf_idf.sum(axis=1)
scoreGood = pd.DataFrame(tf_idf, columns=["score"])
scoreGood["term"] = terms
scoreGood.sort_values(by="score", ascending=False, inplace=True)   # generate TF-IDF scores
# scoreGood.to_csv('../data/scoreGood.csv')

In [18]:
scoreGood.head(50)

,score,term
milk tea boba,538.074948,milk tea boba
taro milk tea,319.699441,taro milk tea
boba milk tea,305.666394,boba milk tea
hokkaido milk tea,217.541442,hokkaido milk tea
jasmine milk tea,200.518574,jasmine milk tea
green milk tea,186.143364,green milk tea
sea salt cream,171.454543,sea salt cream
thai tea boba,168.548062,thai tea boba
rolled ice cream,164.863556,rolled ice cream
oolong milk tea,161.244215,oolong milk tea


For **bad** reviews:

In [19]:
X = vectorizer.fit_transform(lemma_reviewsBad)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
tf_idf = tf_idf.sum(axis=1)
scoreBad = pd.DataFrame(tf_idf, columns=["score"])
scoreBad["term"] = terms
scoreBad.sort_values(by="score", ascending=False, inplace=True)   # generate TF-IDF scores
# scoreBad.to_csv('../data/scoreBad.csv')

In [20]:
scoreBad.head(50)

,score,term
milk tea boba,75.801106,milk tea boba
boba milk tea,53.965356,boba milk tea
taro milk tea,52.992712,taro milk tea
jasmine milk tea,37.663909,jasmine milk tea
green milk tea,34.588488,green milk tea
bubble tea shop,31.220590,bubble tea shop
milk tea tasted,30.431712,milk tea tasted
milk green tea,28.682561,milk green tea
tea tasted like,28.185444,tea tasted like
thai tea boba,27.989017,thai tea boba
